# Computing gradient

Training DNNs requires computing gradient of the loss with respect to the parameters, and this is topic of this notebook.

Your task will be to:

* Implement Evolution Search training of DNNs that does not require backpropagation
* Implement forward and backward functions for Linear and ReQU modules

Goal is to:

* Introduce ingredients necessary to train NN:
    * model
    * loss
    * optimizer
* Introduce different ways of computing or estimating $\frac{\partial{L(w)}}{{\partial w}}$
* Get understanding of Evolution Search and Backpropagation algorithms

Exam:

* For exam you are expected to understand how backpropagation works. Questions might refer to implementation details in PyTorch. See also Resources section for a good video about backpropgation, and how it is implemented in PyTorch.

What's (probably) next:

* Go through each component of the training loop:
    * (Lab 5) Neural Networks practical, part 1: architecture, understanding what is learned
    * (Lab 6, ?) Neural Networks practical part 2: setting up data and loss
    * (Lab 7, ?) Neural Networks practical part 3: optimization

Resources:

* Backprop with focus on PyTorch: https://www.youtube.com/watch?v=ma2KXWblllc (see also other lectures from this series)

* Lecture on backpropagation https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/ (Lecture 8)

* Evolution Search https://eng.uber.com/deep-neuroevolution/, https://arxiv.org/abs/1712.06564

* Matrix derivatives http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf

# SGD requires $\frac{\partial L (w) }{\partial w}$!

<p>
<font size=5>
$$\frac{\partial L (w) }{\partial w} = \frac{1}{K} \sum_{i=1}^{K} \frac{\partial L (x_i, w) }{\partial w}$$
</font>
</p>

<img width=600 src="https://github.com/gmum/nn2018/raw/master/lab/fig/3/sgd.png">

# Setup 

In [20]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import json

import torch
import torch.nn.functional as F

from torch import optim
from torch import nn
from torch.autograd import Variable

from keras.datasets import fashion_mnist
from keras.utils import np_utils

%matplotlib inline
import matplotlib.pylab as plt
import matplotlib as mpl

from torch.autograd import gradcheck

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

# Get FashionMNIST (see 1b_FMNIST.ipynb for data exploration)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Logistic regression needs 2D data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# 0-1 normalization
x_train = x_train / 255.
x_test = x_test / 255.

# Convert to Torch Tensor. Just to avoid boilerplate code later
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

# Use only first 1k examples. Just for notebook to run faster
x_train, y_train = x_train[0:1000], y_train[0:1000]
x_test, y_test = x_test[0:1000], y_test[0:1000]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
## Some helper functions

def get_model_weights(model):
    params = model.state_dict()
    p_order = [p[0] for p in sorted(model.named_parameters())]
    return np.concatenate([params[w].cpu().numpy().reshape(-1, ) for w in p_order])
    
def set_model_weights(model, w):
    params = model.state_dict()  
    p_order = [p[0] for p in sorted(model.named_parameters())]
    id = 0
    for p in p_order:
        shape = params[p].shape
        D = np.prod(shape)
        params[p].copy_(torch.from_numpy(w[id:id + D].reshape(shape)))
        id += D
        
def L(w, loss, model, batch_size=10, x_tr=x_train, y_tr=y_train):
    set_model_weights(model, w)
    cost = 0.
    n_examples, n_features = x_tr.size()
    num_batches = n_examples // batch_size
    for k in range(num_batches): 
        start, end = k * batch_size, (k + 1) * batch_size
        x = Variable(x_tr[start:end], requires_grad=False)
        y = Variable(y_tr[start:end], requires_grad=False)
        fx = model.forward(x)
        cost += loss.forward(fx, y)
    return cost.data.numpy()[0] / num_batches

# Training loop in PyTorch

In this section we walk through implementation of a traditional training loop  in PyTorch.

Necessary ingredients to train a neural network are:
    * model
    * loss
    * optimizer
    
<img width=400 src="https://github.com/gmum/nn2018/raw/master/lab/fig/4/smoothie.png">

Ref:

https://github.com/vinhkhuc/PyTorch-Mini-Tutorials/blob/master/3_neural_net.py

In [23]:
def build_logreg(input_dim, output_dim, weight_module=torch.nn.Linear):
    model = torch.nn.Sequential()
    model.add_module("linear_2", weight_module(input_dim, output_dim, bias=False))
    return model

In [24]:
def build_model(input_dim, output_dim, nonlinearity=torch.nn.functional.sigmoid, weight_module=torch.nn.Linear):
    class Nonlinearity(torch.nn.Module):
        def forward(self, input):
            return nonlinearity(input)
    
    model = torch.nn.Sequential()
    model.add_module("linear_1", weight_module(input_dim, 512, bias=False))
    model.add_module("nonlinearity_1", Nonlinearity())
    model.add_module("linear_2", weight_module(512, output_dim, bias=False))
    return model

In [25]:
def sgd_step(model, loss, x_val, y_val, lr=0.1, momentum=0.9):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward(retain_graph=True)

    # Update parameters
    optimizer.step()

    return output.data[0]

In [26]:
def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

In [27]:
def train(model, loss, step_fnc):
    torch.manual_seed(42)
    n_examples, n_features = x_train.size()
    n_classes = 10
    batch_size = 100
    history = []
    for i in tqdm.tqdm(range(100), total=100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += step_fnc(model, loss, x_train[start:end], y_train[start:end])
        predY = predict(model, x_test)
        print(("Epoch %d, cost = %f, acc = %.2f%%"
              % (i + 1, cost / num_batches, 100. * np.mean(predY == y_test.numpy()))))
        
        history.append(cost)
        
    return np.mean(predY == y_test.numpy()), history

## Run training

In [29]:
model = build_model(784, 10)
loss = torch.nn.CrossEntropyLoss(size_average=True)
train(step_fnc=sgd_step, loss=loss, model=model)

  1%|          | 1/100 [00:00<00:30,  3.24it/s]

Epoch 1, cost = 2.365059, acc = 9.50%
Epoch 2, cost = 2.273749, acc = 9.80%

  2%|▏         | 2/100 [00:00<00:24,  4.05it/s]


Epoch 3, cost = 2.156826, acc = 16.80%
Epoch 4, cost = 2.028474, acc = 30.40%

  4%|▍         | 4/100 [00:00<00:19,  4.95it/s]

...

 97%|█████████▋| 97/100 [00:09<00:00, 12.31it/s]


Epoch 98, cost = 0.457565, acc = 77.90%
Epoch 99, cost = 0.455014, acc = 77.90%

 99%|█████████▉| 99/100 [00:09<00:00, 12.74it/s]


Epoch 100, cost = 0.452495, acc = 77.90%

(0.779,
 [23.65058732032776,
  22.73749351501465,
  21.568257093429565,
  20.284743905067444,
  18.94912815093994,
  17.65986692905426,
  16.490514278411865,
  15.467840433120728,
  14.585906982421875,
  13.825463056564331,
  13.165529131889343,
  12.587876796722412,
  12.07787811756134,
  11.624071955680847,
  11.217470169067383,
  10.8509521484375,
  10.51879495382309,
  10.216351449489594,
  9.93979287147522,
  9.685938358306885,
  9.452118039131165,
  9.236069560050964,
  9.035862863063812,
  8.849831700325012,
  8.676527976989746,
  8.51468962430954,
  8.363200962543488,
  8.221074998378754,
  8.087433516979218,
  7.961495280265808,
  7.842560291290283,
  7.729999780654907,
  7.623251020908356,
  7.521805226802826,
  7.4252079129219055,
  7.33304899930954,
  7.2449570298194885,
  7.160598874092102,
  7.079673588275909,
  7.0019097328186035,
  6.92706161737442,
  6.854906857013702,
  6.785244703292847,
  6.71789425611496,
  6.652690172195435,
  6.589481890201569,
  6.52813559770584

# Estimating $\frac{\partial L (w) }{\partial w}$

In this section we will implement different gradient computation functions

## Finite Difference
<p>
<font size=4>
Assuming we know direction we want to optimize in, we can approximate derivative simply by calculating

$$\langle \frac{\partial L (w) }{\partial w}, \vec{v} \rangle \approx \frac{L(w) - L(w + v)}{|\vec{v}|}$$


, where $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

<p>
<font size=4>
As we will see later PyTorch uses Finite Difference method in gradient checks of automatic diffentiation modules.
</font>
</p>

## Evolution Search

<p>
Intuitively, Evolution Search (ES) approximates gradient by computing multiple times finite difference. Importantly it works for non-differentiable costs as well.
</p>

<p>
<font size=4>
$$ \frac{\partial L (w) }{\partial w} \approx \frac{1}{N \sigma} \sum \vec{v} ( L(w + \vec{v}) - L(w) )$$


, where $\epsilon$ is a normally distributed vector sampled from gauss of standard deviation $\sigma$, and $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

<p>
<font size=4>
Note: while it is recommended to not use minibatching in ES, for simplicity we will.
</font>
</p>

## Implement Evolution Search

In [31]:
def ES_grad(model, loss, x_tr, y_tr, sigma=0.001, N=100):
    """
    Estimate dL/dw using Evolution Search.
    
    Params
    ------
    model: torch.nn.Model
        Model to take optimization step on
    loss: torch.nn.Module
        Loss function to optimize
    x_tr: torch.Tensor, (n_examples, n_features)
        Batch to compute gradient over
    y_tr: torch.Tensor, (n_examples, )
        Batch to compute gradient over
    sigma: float
    
    N: int
        
    Returns
    -------
    g: np.vector, size: (D, )
        Gradient of loss with respect to model's weights
    """
    grad = 0
    init_w = get_model_weights(model)
    rng = np.random.RandomState()
    base_loss = L(w=init_w, model=model, loss=loss, x_tr=x_tr, y_tr=y_tr)
    for _ in tqdm.tqdm(range(N), total=N):
        v = ?? # Hint: use rng
        reward = ?? # Hint: use L
        grad += ?? # Hint: see equation
    return ?? # Hint: normalize!

SyntaxError: invalid syntax (<ipython-input-31-1f1613c57dc5>, line 29)

In [ ]:
# If code is slow, you can use build_logreg !
# model = build_logreg(784, 10) 
model = build_model(784, 10)
loss = torch.nn.CrossEntropyLoss(size_average=True)
init_w = get_model_weights(model)

In [ ]:
down = 0
for _ in range(10):
    loss_before = L(w=init_w, model=model, loss=loss)
    set_model_weights(model=model, w=init_w)
    v = ES_grad(model, loss, x_tr=x_train, y_tr=y_train, sigma=0.002, N=50)
    loss_after = L(w=init_w - 0.1*v, model=model, loss=loss)
    if loss_before > loss_after:
        down += 1
        print("Went down!")
print("{}/{}".format(down, 10))
# This code is stochastic, so of course sometimes this assert can fail. 
assert down > 6

## Implement own module

PyTorch enable defining own Modules. Here we will implement Linear and ReQu modules ourselves, to understand better how backpropagation works. 

Below you can find an example Module implementing the classial ReLU nonlinearity.

Ref: https://github.com/jcjohnson/pytorch-examples

In [32]:
class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  def forward(self, input):
    """
    In the forward pass we receive a Tensor containing the input and return a
    Tensor containing the output. You can cache arbitrary Tensors for use in the
    backward pass using the save_for_backward method.
    """
    self.save_for_backward(input)
    return input.clamp(min=0)

  def backward(self, grad_output):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    input, = self.saved_tensors
    grad_input = grad_output.clone()
    print(grad_input)
    grad_input[input < 0] = 0
    return grad_input

In [39]:
# Gradient check passes!
input = (Variable(torch.randn(2,4).double(), requires_grad=True),)
assert(gradcheck(MyReLU(), input, eps=1e-6, atol=1e-4))


 1  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 1  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  1  0  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  1  0  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  1  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  1  0
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  1
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  1
 0  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 1  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 1  0  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  1  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  1  0  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  0  1  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  0  1  0
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  0  0  1
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  0  0  1
[torch.DoubleTensor of size 2x4]


 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 2x4

In [ ]:
model = build_model(n_features, n_classes, nonlinearity=MyReLU())
loss = torch.nn.CrossEntropyLoss(size_average=True)
_, H_relu = train(step_fnc=sgd_step, loss=loss, model=model)

### Implement “ReQU” unit

<p>
<font size=4>
Here, we’ll implement a made-up activation function that we’ll call the Rectified Quadratic Unit
(ReQU). Like the sigmoid and ReLU and several others, it is applied element-wise to all its
inputs:
</font>
</p>

<p>
<font size=4>
$$z_i = I[x_i > 0] x_i ^ 2$$
</font>
</p>

(Note, exercise is taken from  https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/practicals/practical4.pdf)

In [ ]:
# Hint: use ReLU implementation as a template

class ReQU(torch.autograd.Function):
  def forward(self, input):
    self.save_for_backward(input)
    ??

  def backward(self, grad_output):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    input, = self.saved_tensors
    ??

In [ ]:
## GradCheck
input = (Variable(torch.randn(20,20).double(), requires_grad=True),)
assert(gradcheck(ReQU(), input, eps=1e-6, atol=1e-4))

In [ ]:
model = build_model(n_features, n_classes, nonlinearity=ReQU())
loss = torch.nn.CrossEntropyLoss(size_average=True)
_, H_requ = train(step_fnc=sgd_step, loss=loss, model=model)

In [ ]:
### Comparing ReLU and ReQU
plt.plot(H_relu, label="ReLU")
plt.plot(H_requ, label="ReQU")
plt.ylim([0, 10])
plt.legend()

# Tests

In [ ]:
result = {}

input = (Variable(torch.randn(20,20).double(), requires_grad=True),)
result['requ'] = 1.0*int(gradcheck(ReQU(), input, eps=1e-6, atol=1e-4))

# If code is slow, you can use build_logreg !
model = build_logreg(784, 10) 
loss = torch.nn.CrossEntropyLoss(size_average=True)
init_w = get_model_weights(model)
down = 0
for _ in range(10):
    loss_before = L(w=init_w, model=model, loss=loss)
    set_model_weights(model=model, w=init_w)
    v = ES_grad(model, loss, x_tr=x_train, y_tr=y_train, sigma=0.002, N=50)
    loss_after = L(w=init_w - 0.1*v, model=model, loss=loss)
    if loss_before > loss_after:
        down += 1
# This code is stochastic, so of course sometimes this assert can fail. 
result['ES'] = int(down > 6)

In [ ]:
json.dump(result, open("4a_computing_gradient.json", "w"))